In [12]:
import pandas as pd
from selenium import webdriver
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
import json
import multiprocessing as mp
from multiprocessing import freeze_support

In [13]:
dirr = 'C:/Users/shic/Desktop/kb_land_req_text.csv'
dataa = pd.read_csv(dirr).reset_index(drop=True)

In [55]:
dataa.columns

Index(['year', 'month', 'region_b', 'region_s', 'req_text'], dtype='object')

In [225]:
tmp_list = []
df_list = []
for datum in dataa.iterrows():
    year  = str(datum[1]['year'])
    month = str(datum[1]['month']) 
    
    if re.search("graphYn1",datum[1]['req_text']) is None:
            no_return_dataframe = pd.DataFrame({'년' : year,
                                                '월' : month,
                                                '시/도' : datum[1]['region_b'],
                                                '시/군/구': datum[1]['region_s'],
                                                '지수_date' : 0,
                                                '지수'   : 0,
                                                'len_a' : 0,
                                                'len_b' : 0
            },index=[0])
            df_list.append(no_return_dataframe)
    else:
        ret_a = re.search("var graphYn1 = [$](.+?);", datum[1]['req_text'], re.S).group(1)
        ret_a = ret_a.replace(".trim('[", "").replace("') == '' ? false: true", "").replace('Date.UTC(', '')
        
        date_list_jisu = []
        data_list_jisu = []

        jisu_split = ret_a.split('], [')

        for jisu in jisu_split:
            jisu_parser = jisu.split('), ')

            jisu_date_parser = jisu_parser[0].replace(' ','').split(',')
            
            date_list_jisu.append(','.join([
                jisu_date_parser[0], str(int(jisu_date_parser[1]) + 1), jisu_date_parser[2]
            ]))
            data_list_jisu.append(jisu_parser[1].replace(']','').replace(',',''))
            
        date_list_rate = []
        data_list_rate = []
        
        if re.search("var graphYn2 = [$].trim\(.{1,2}\)", datum[1]['req_text'], re.S) is not None:
            date_list_rate = np.repeat(0,len(date_list_jisu)).tolist()
            data_list_rate = np.repeat(0,len(data_list_jisu)).tolist()
        else:
            ret_b = re.search('var graphYn2 = [$](.+?);', datum[1]['req_text'], re.S).group(1)
            ret_b = ret_b.replace(".trim('[", "").replace("') == '' ? false: true", "").replace('Date.UTC(', '')
        
            rate_split = ret_b.split('], [')
        
            for rate in rate_split:
                rate_parser = rate.split('), ')

                rate_date_parser = rate_parser[0].replace(' ','').split(',')

                date_list_rate.append(','.join([
                    rate_date_parser[0], str(int(rate_date_parser[1]) + 1), rate_date_parser[2]
                    ]))
                
                data_list_rate.append(rate_parser[1].replace(']','').replace(',',''))
                
        return_dataframe = pd.DataFrame({
                '년'      : year,
                '월'     : month,
                '시/도'  : datum[1]['region_b'],
                '시/군/구': datum[1]['region_s'],
                '지수_date'   : date_list_jisu,
                '지수'   : data_list_jisu,
                'len_a' : len(date_list_rate),
                'len_b' : len(data_list_rate)
            })
        df_list.append(return_dataframe)

In [226]:
df_bind = pd.concat(df_list,axis=0)

In [227]:
print(df_bind['len_a'].unique())
print(df_bind['len_b'].unique())

[47  0 31 35 46 39 44 48  3  7 11 16 20 24 28 32 37 40 45  2  6 10 14 18
 23 27 36  5  9 22 43]
[47  0 31 35 46 39 44 48  3  7 11 16 20 24 28 32 37 40 45  2  6 10 14 18
 23 27 36  5  9 22 43]


In [228]:
len(data_list_rate)

46